# Random forests to predict ScientificName and determine genomes that impact the accuracy of that classification most

In [2]:
setwd("..")

In [50]:
library(readr)
library(dplyr)
library(purrr)
library(tidyr)
library(tibble)
library(ggplot2)
library(caret)
library(ranger)
source("scripts/utils.R")

In [ ]:
# set.seed(1) # note I forgot to run this for the current output. sigh.

In [3]:
# change default figure size
options(repr.plot.width=15, repr.plot.height=7)
# disable scientific notation (for plot axes)
options(scipen = 999)

## Read in gather results and sample metadata

In [4]:
# separate empty files from populated files
files <- Sys.glob("results/*gather.csv")
empty_files <- character()
populated_files <- character()
for(i in 1:length(files)){
    # check and see if the file is empty, e.g. has not gather matches
    file_size <- file.size(files[i])
    if(file_size == 0){
        empty_files = c(empty_files, files[i])
    } else {
        populated_files = c(populated_files, files[i])
    }
}

In [8]:
# read in populated files
gather_results <- populated_files %>%
  map_dfr(read_gather)

In [9]:
# combine populated files with empty files
query_name <- gsub("\\.gather\\.csv", "", basename(empty_files))
query_name <- as.data.frame(query_name)
gather_results <- bind_rows(gather_results, query_name)

In [10]:
# join gather results with metadata
runinfo <- read_csv("all.runinfo.csv")
gather_results <- left_join(gather_results, runinfo, by = c("query_name" = "Run"))

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 15190 Columns: 47

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (29): Run, AssemblyName, download_path, Experiment, LibraryName, Librar...
dbl  (10): spots, bases, spots_with_mates, avgLength, size_MB, InsertSize, I...
lgl   (6): g1k_pop_code, source, g1k_analysis_group, Disease, Affection_Stat...
dttm  (2): ReleaseDate, LoadDate


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



## Filter to biomes that have many observations and have names that make sense in a metagenome context

In [58]:
# table(gather_results$ScientificName) # must have at least 10 and not be a species name to be kept
# and not be something amorphous like "environmental samples" or "mixed sample"

In [15]:
filter_set <-c("human gut metagenome", "gut metagenome", "metagenome", "soil metagenome", "human metagenome", 
               "marine metagenome", "wastewater metagenome", "pig gut metagenome", "sediment metagenome", 
               "feces metagenome", "freshwater metagenome", "bovine gut metagenome", "mouse gut metagenome", 
               "human skin metagenome", "seawater metagenome", "human oral metagenome", "aquatic metagenome", 
               "rhizosphere metagenome", "sludge metagenome", "chicken gut metagenome", "activated sludge metagenome",
               "peat metagenome", "lake water metagenome", "air metagenome", "marine plankton metagenome", 
               "viral metagenome", "root metagenome", "bat metagenome", "biofilm metagenome", "biofilter metagenome",
               "activated carbon metagenome", "algae metagenome", "anaerobic digester metagenome", "annelid metagenome",
               "biogas fermenter metagenome", "bioreactor metagenome", "bioreactor sludge metagenome", "bird metagenome",
               "blood metagenome", "bovine metagenome", "canine metagenome", "cetacean metagenome", "ciliate metagenome",
               "coral metagenome", "compost metagenome", "coral reef metagenome", "crab metagenome", "crustacean metagenome",
               "drinking water metagenome", "dust metagenome", "echinoderm metagenome", "epibiont metagenome",
               "estuary metagenome", "fermentation metagenome", "fish gut metagenome", "fish metagenome", 
               "food contamination metagenome", "food fermentation metagenome", "food metagenome", 
               "food production metagenome", "fossil metagenome", "fungus metagenome", "glacier metagenome", 
               "groundwater metagenome", "halite metagenome", "horse metagenome", "hospital metagenome",
               "hot springs metagenome", "human blood metagenome", "human eye metagenome", "human feces metagenome",
               "human lung metagenome", "human milk metagenome", "human nasopharyngeal metagenome", 
               "human saliva metagenome", "human skeleton metagenome", "human tracheal metagenome",
               "human urinary tract metagenome", "human vaginal metagenome", "human viral metagenome", 
               "hydrothermal vent metagenome", "hypersaline lake metagenome", "indoor metagenome",
               "industrial waste metagenome", "insect gut metagenome", "insect metagenome", "invertebrate metagenome",
               "lichen metagenome", "lung metagenome", "mangrove metagenome", "manure metagenome", "marine sediment metagenome",
               "metagenomes", "microbial mat metagenome", "milk metagenome", "mine drainage metagenome", "mine tailings metagenome",
               "mixed culture metagenome", "mollusc metagenome", "money metagenome", "mosquito metagenome", "mouse metagenome",
               "oral metagenome", "pig metagenome", "plant metagenome", "pond metagenome", "rat gut metagenome", 
               "reproductive system metagenome", "respiratory tract metagenome", "riverine metagenome", "rock metagenome",
               "rodent metagenome", "salt lake metagenome", "salt marsh metagenome", "sand metagenome", "sea squirt metagenome",
               "Severe acute respiratory syndrome coronavirus 2", "sheep gut metagenome", "shrimp gut metagenome",
               "soil crust metagenome", "sponge metagenome", "starfish metagenome", "subsurface metagenome", 
               "symbiont metagenome", "synthetic metagenome", "termite gut metagenome", "terrestrial metagenome", 
               "tick metagenome", "uncultured human fecal virus", "uncultured virus", "unidentified",
               "upper respiratory tract metagenome", "urinary tract metagenome", "urine metagenome", "vaginal metagenome",
               "wetland metagenome", "whole organism metagenome", "wine metagenome")

In [16]:
# make sure set is typo free and doesn't contain duplicates
length(filter_set)
length(unique(filter_set))
table(filter_set %in% gather_results$ScientificName)
filter_set[!filter_set %in% gather_results$ScientificName]

[1] 138

[1] 138


TRUE 
 138 

character(0)

In [17]:
# filter gather_results to only contain observations in filter_set
gather_results <- gather_results %>%
  filter(ScientificName %in% filter_set)

In [18]:
nrow(gather_results)

[1] 2058211

In [19]:
# number of samples (metagenomes)
gather_results %>%
  select(query_name) %>%
  distinct() %>%
  nrow()

# number of genomes observed in those samples (metagenomes)
gather_results %>%
  select(name) %>%
  distinct() %>%
  nrow()

[1] 8716

[1] 76379

_NB_ data are still high dimensional -- we have many more variables (genomes) than we have samples (metagenomes)  
Variable selection is probably recommended 

## Format for random forests

In [26]:
# samples (metagenomes) need to be rownames, names (genomes) need to be columns, class (ScientificName) can be a column at end of df for now
gather_formatted <- gather_results %>%
  mutate(accession = gsub(" .*", "", name)) %>% # make genome names more friendly
  select(query_name, ScientificName, accession, f_unique_to_query) %>%
  distinct() %>% # somehow duplicates snuck in?
  pivot_wider(id_cols = c("query_name", "ScientificName"), names_from = accession, values_from = f_unique_to_query)

In [28]:
gather_formatted[is.na(gather_formatted)] <- 0 # replace NAs with 0s

In [30]:
gather_formatted[1:5, 1:5]
dim(gather_formatted)

query_name,ScientificName,GCF_003478165.1,GCF_012271835.1,GCF_003459645.1
<chr>,<chr>,<dbl>,<dbl>,<dbl>
DRR014176,human metagenome,0.034210526,0.0215789474,0.020526316
DRR025071,pig gut metagenome,0.000000000,0.0000000000,0.000000000
DRR033608,human oral metagenome,0.000000000,0.0000000000,0.000000000
DRR042304,human gut metagenome,0.000000000,0.0004482295,0.007619901
DRR042358,human gut metagenome,0.001295337,0.0012953368,0.000000000


[1]  8716 76381

In [57]:
# filtering on biomes was done before collapsing; make sure every biome has enough observations
tmp <- gather_formatted %>%
  group_by(ScientificName) %>% 
  tally() %>%
  filter(n > 10) %>%
  arrange(desc(n))

## Generate training and testing set

In [43]:
# filter to ScientificNames with at least 10 observations
gather_formatted2 <- gather_formatted %>%
  filter(ScientificName %in% tmp$ScientificName)

biome <- gather_formatted2$ScientificName

gather_formatted2 <- gather_formatted2 %>%
  select(-ScientificName) %>%
  column_to_rownames("query_name")

## semi-randomly create test set
train_indx <- createDataPartition(biome, p = .7, list = FALSE, times = 1)

train_df <- gather_formatted2[train_indx, ]
test_df <-  gather_formatted2[-train_indx, ]

# remove testdata from training data
train_biome <- biome[train_indx]
test_biome  <- biome[-train_indx]

## Run variable selection on training data

### Vita variable selection via pomona package wrappers 

Use functions to avoid package installation -- see `scripts/utils.R`

In [51]:
vita <- var.sel.vita(x = train_df, y = train_biome, p.t = 0.05,
                     ntree = 5000, mtry.prop = 0.2, nodesize.prop = 0.1,
                     no.threads = 3, 
                     method = "ranger", type = "classification")

Growing trees.. Progress: 0%. Estimated remaining time: 2 hours, 27 minutes, 36 seconds.
Growing trees.. Progress: 1%. Estimated remaining time: 2 hours, 27 minutes, 4 seconds.
Growing trees.. Progress: 1%. Estimated remaining time: 2 hours, 25 minutes, 21 seconds.
Growing trees.. Progress: 1%. Estimated remaining time: 2 hours, 25 minutes, 6 seconds.
Growing trees.. Progress: 2%. Estimated remaining time: 2 hours, 22 minutes, 15 seconds.
Growing trees.. Progress: 2%. Estimated remaining time: 2 hours, 21 minutes, 6 seconds.
Growing trees.. Progress: 3%. Estimated remaining time: 2 hours, 19 minutes, 59 seconds.
Growing trees.. Progress: 3%. Estimated remaining time: 2 hours, 19 minutes, 27 seconds.
Growing trees.. Progress: 3%. Estimated remaining time: 2 hours, 18 minutes, 32 seconds.
Growing trees.. Progress: 4%. Estimated remaining time: 2 hours, 18 minutes, 7 seconds.
Growing trees.. Progress: 4%. Estimated remaining time: 2 hours, 16 minutes, 36 seconds.
Growing trees.. Progress:

In [52]:
# save the model, just in case
saveRDS(vita, "outputs/gather10k_rf/scientificname_vita_varsel.RDS")

In [53]:
## write predictive accessions
var <- vita$var   # separate out selected predictive accessions
write.table(var, "outputs/gather10k_rf/scientificname_vita_varsel_vars.txt", quote = F, col.names = F, row.names = F)

In [55]:
# write out full training and testing sets as dataframes 

# first, re-combine class vector with
train_df <- train_df %>%
  mutate(ScientificName = train_biome) %>%
  rownames_to_column("query_name")

test_df <- test_df %>%
  mutate(ScientificName = test_biome) %>%
  rownames_to_column("query_name")

# write full data frames
write_csv(train_df, "outputs/gather10k_rf/scientificname_full_training_data.csv")
write_csv(test_df, "outputs/gather10k_rf/scientificname_full_testing_data.csv")

In [56]:
# filter training and testing data to predictive accessions and write to file
train_df_filt <- train_df[ , colnames(train_df) %in% c('query_name', 'ScientificName', var)] # subset to selectec vars
write_csv(train_df_filt, "outputs/gather10k_rf/scientificname_filt_training_data.csv")

test_df_filt <- test_df[ , colnames(test_df) %in% c('query_name', 'ScientificName', var)] # subset to selectec vars
write_csv(test_df_filt, "outputs/gather10k_rf/scientificname_filt_testing_data.csv")